In [1]:
# !pip install transformers

In [2]:
# !pip install tiktoken

In [3]:
import kagglehub
import torch 
from transformers import AutoTokenizer 
from transformers.models.gemma3 import Gemma3ForCausalLM


In [4]:

#Model Setup: Downloads the Gemma 3 (1 billion parameter, instruction-tuned version) directly from Kaggle Hub.
GEMMA_PATH = "google/gemma-3-1b-pt"


In [6]:
# !pip install -q huggingface_hub


In [2]:
from huggingface_hub import login
import os

token = os.getenv("HUGGINGFACE_TOKEN")

login(token)


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
print(f"Device in use: {device}")


Device in use: cuda


In [12]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM


ckpt = "google/gemma-3-1b-pt"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [13]:
prompt = "Eiffel tower is located in"
model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

input_len = model_inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**model_inputs, max_new_tokens=50, do_sample=False)
    generation = generation[0][input_len:]

decoded = tokenizer.decode(generation, skip_special_tokens=True)
print(decoded)

 the heart of Paris, France.The Eiffel Tower is a symbol of Paris and France.The Eiffel Tower is a symbol of Paris and France.The Eiffel Tower is a symbol of Paris and France.The Eiffel Tower is a symbol


Device in use: cuda


In [14]:
def create_prompt(user_question):
    return f"""<start_of_turn>user\n{user_question}<end_of_turn>\n<start_of_turn>model"""


generation_config = {
    "max_new_tokens": 1024, 
    "temperature": 0.5, 
    "do_sample": True, #greedy decoding to make it fastere do_sample = false 
    "top_p": 0.7, 
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.eos_token_id
}

def ask_gemma(question): 
    prompt = create_prompt(question)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, **generation_config)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)

In [15]:
user_question = "How can we get potable Water Gemma 3?"
ask_gemma(user_question)

user
How can we get potable Water Gemma 3?
model
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we get potable Water Gemma 3?clusively
How can we 

In [17]:
import pandas as pd

In [20]:
df = pd.read_csv('/home/cse/Desktop/VOIP_VISHING/REPOS/conversation-prediction/FINAL_DATASET2.csv',encoding='latin-1')

In [21]:
df

,CONVERSATION_ID,CONVERSATION_STEP,TEXT,CONTEXT,LABEL,Unnamed: 5,Unnamed: 6
0,0,1,"Hello, this is [Your Name]'s personal assistan...",Standard opening exchange,neutral,NaN,NaN
1,0,2,"Hi, I'm Sam. I saw an ad about a photography w...",Expresses interest,neutral,NaN,NaN
2,0,3,"Hi Sam, it's great to hear of your interest in...",Assistant is open and encouraging,neutral,NaN,NaN
3,0,4,Thanks! I was wondering about the skill level ...,Addresses the concern directly,neutral,NaN,NaN
4,0,5,The workshop is designed to accommodate all sk...,Addresses the concern directly,neutral,NaN,NaN
...,...,...,...,...,...,...,...
962,75,27,Understood. Weâll try to expedite the process.,Acknowledging and offering assistance,neutral,NaN,NaN
963,75,28,"If I delete the app, I wonât get messages fr...",Clarifying concerns about app deletion,neutral,NaN,NaN
964,75,29,"No, deleting the app wonât affect our commun...",Reassuring customer,neutral,NaN,NaN
965,75,30,"Alright, proceed and call me back in 10 minutes.",Requesting follow-up,neutral,NaN,NaN


In [22]:
# Group by CONVERSATION_ID and create full conversations
df['FULL_CONVERSATION'] = df.groupby(['CONVERSATION_ID'])['TEXT'].transform(lambda x: ' '.join(x))

# Create partial conversations (excluding the last step)
df['PARTIAL_CONVERSATION'] = df.groupby(['CONVERSATION_ID'])['TEXT'].transform(lambda x: ' '.join(x[:-1]))

In [23]:
df

,CONVERSATION_ID,CONVERSATION_STEP,TEXT,CONTEXT,LABEL,Unnamed: 5,Unnamed: 6,FULL_CONVERSATION,PARTIAL_CONVERSATION
0,0,1,"Hello, this is [Your Name]'s personal assistan...",Standard opening exchange,neutral,NaN,NaN,"Hello, this is [Your Name]'s personal assistan...","Hello, this is [Your Name]'s personal assistan..."
1,0,2,"Hi, I'm Sam. I saw an ad about a photography w...",Expresses interest,neutral,NaN,NaN,"Hello, this is [Your Name]'s personal assistan...","Hello, this is [Your Name]'s personal assistan..."
2,0,3,"Hi Sam, it's great to hear of your interest in...",Assistant is open and encouraging,neutral,NaN,NaN,"Hello, this is [Your Name]'s personal assistan...","Hello, this is [Your Name]'s personal assistan..."
3,0,4,Thanks! I was wondering about the skill level ...,Addresses the concern directly,neutral,NaN,NaN,"Hello, this is [Your Name]'s personal assistan...","Hello, this is [Your Name]'s personal assistan..."
4,0,5,The workshop is designed to accommodate all sk...,Addresses the concern directly,neutral,NaN,NaN,"Hello, this is [Your Name]'s personal assistan...","Hello, this is [Your Name]'s personal assistan..."
...,...,...,...,...,...,...,...,...,...
962,75,27,Understood. Weâll try to expedite the process.,Acknowledging and offering assistance,neutral,NaN,NaN,"Hello, this is your manager. I've been trying ...","Hello, this is your manager. I've been trying ..."
963,75,28,"If I delete the app, I wonât get messages fr...",Clarifying concerns about app deletion,neutral,NaN,NaN,"Hello, this is your manager. I've been trying ...","Hello, this is your manager. I've been trying ..."
964,75,29,"No, deleting the app wonât affect our commun...",Reassuring customer,neutral,NaN,NaN,"Hello, this is your manager. I've been trying ...","Hello, this is your manager. I've been trying ..."
965,75,30,"Alright, proceed and call me back in 10 minutes.",Requesting follow-up,neutral,NaN,NaN,"Hello, this is your manager. I've been trying ...","Hello, this is your manager. I've been trying ..."


In [24]:
combined_list = [
    f"{partial}:() {full}()" 
    for partial, full in zip(df['PARTIAL_CONVERSATION'], df['FULL_CONVERSATION'])
]


In [25]:
Train_dataset = []
    
for index, row in df.iterrows():
    FULL_CONVERSATION, PARTIAL_CONVERSATION = row['FULL_CONVERSATION'], row['PARTIAL_CONVERSATION']
    template = (f"FullConversation:\n{FULL_CONVERSATION}\n\PartialConversation:\n{PARTIAL_CONVERSATION}")
    if len(template) < 850:
        Train_dataset.append(template)

<>:5: SyntaxWarning: invalid escape sequence '\P'
<>:5: SyntaxWarning: invalid escape sequence '\P'
/tmp/ipykernel_83608/2219208535.py:5: SyntaxWarning: invalid escape sequence '\P'
  template = (f"FullConversation:\n{FULL_CONVERSATION}\n\PartialConversation:\n{PARTIAL_CONVERSATION}")


In [31]:
len(Train_dataset)

5

In [32]:
len(combined_list)

967

In [27]:
gemma_lm.backbone.enable_lora(rank=4)

NameError: name 'gemma_lm' is not defined